In [1]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import xgboost as xgb
from scipy import stats

def XGBoost(X, y):
    # データを訓練用とテスト用に分割
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # 訓練データをさらに訓練用と検証用に分割
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=0)

    # モデルのパラメータを設定(CPU)
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': 0.1,
        'max_depth': 4,
        'seed': 0
    }
    
    """   # モデルのパラメータを設定(GPU)
    params = {
        'objective': 'multi:softmax',
        'eval_metric': 'mlogloss',
        'num_class' : 4,
        'eta': 0.1,
        'max_depth': 12,
        'seed': 0,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
    }"""

    # 訓練データと検証データをXGBoostのDMatrix形式に変換
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # 訓練データと検証データのセットをリストに格納
    evals = [(dtrain, 'train'), (dval, 'eval')]

    # モデルを訓練
    bst = xgb.train(params, dtrain, num_boost_round=10000, evals=evals, early_stopping_rounds=100)

    # テストデータをDMatrix形式に変換
    dtest = xgb.DMatrix(X_test)

    # 訓練データで予測
    y_train_pred = bst.predict(dtrain)

    # テストデータで予測
    y_pred = bst.predict(dtest)

    # 評価
    train_accuracy = accuracy_score(y_train, y_train_pred.round())
    print(f"Training Accuracy: {train_accuracy*100:.2f}%")

    # 評価
    test_accuracy = accuracy_score(y_test, y_pred.round())
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    return

def normalize_per_state(df, start, end):
    scaler = StandardScaler()
    df.iloc[start:end, :] = scaler.fit_transform(df.iloc[start:end, :])
    return df

In [2]:
raw = 0 #rawデータならば１、Bandなら０,

# 1秒ごとのデータに分割
n_samples_per_second = 256  # 256Hzのサンプリングレート
total_seconds = 10  # 全体の秒数

# データの読み込み
if(raw):
    df = pd.read_csv('Raw.csv')
else:
    df = pd.read_csv('Band.csv')
    #df = pd.read_csv('emotion.csv')
    
# チャンネルごとに正規化
scaler = StandardScaler()
for column in df.columns[:-1]:  # 'State'列を除くすべての列
    df[column] = scaler.fit_transform(df[column].values.reshape(-1, 1))

In [3]:
#１秒ごとに分割
# ラベルの作成（5秒閉じ、5秒開け）
labels_close = [0] * 5 *64 
labels_open = [1] * 5 * 64
labels = labels_close + labels_open

#labels = df.iloc[:, -1]

#14chデータ
n_samples_per_second = 4
total_seconds = 10  # 全体の秒数
#total_seconds = 3242  # 全体の秒数

# データとラベルを1秒ごとに分割
#X = np.array([df.iloc[i*n_samples_per_second:(i+1)*n_samples_per_second, :-1].values for i in range(total_seconds)])

X = np.array([df.iloc[i*n_samples_per_second:(i+1)*n_samples_per_second, :-1].values for i in range(total_seconds*64)])
#labels = np.array([stats.mode(y[i*n_samples_per_second:(i+1)*n_samples_per_second])[0][0] for i in range(total_seconds*64)])


# 1秒ごとのデータの平均を取得
X = X.mean(axis=1)

#XGBoost
XGBoost(X, labels)

#plot
#plot_importance(raw, model1)

[0]	train-logloss:0.63506	eval-logloss:0.64232
[1]	train-logloss:0.59760	eval-logloss:0.60419
[2]	train-logloss:0.55437	eval-logloss:0.57080
[3]	train-logloss:0.52854	eval-logloss:0.54907
[4]	train-logloss:0.49344	eval-logloss:0.51870
[5]	train-logloss:0.47033	eval-logloss:0.49451
[6]	train-logloss:0.44095	eval-logloss:0.46910
[7]	train-logloss:0.42159	eval-logloss:0.44607
[8]	train-logloss:0.40792	eval-logloss:0.43527
[9]	train-logloss:0.37936	eval-logloss:0.40894
[10]	train-logloss:0.35775	eval-logloss:0.39122
[11]	train-logloss:0.33384	eval-logloss:0.37032
[12]	train-logloss:0.30958	eval-logloss:0.34671
[13]	train-logloss:0.29892	eval-logloss:0.33413
[14]	train-logloss:0.28034	eval-logloss:0.31534
[15]	train-logloss:0.27103	eval-logloss:0.30914
[16]	train-logloss:0.25992	eval-logloss:0.30014
[17]	train-logloss:0.25051	eval-logloss:0.29134
[18]	train-logloss:0.24293	eval-logloss:0.28302
[19]	train-logloss:0.23498	eval-logloss:0.27279
[20]	train-logloss:0.22914	eval-logloss:0.26973
[2

[170]	train-logloss:0.01487	eval-logloss:0.06796
[171]	train-logloss:0.01481	eval-logloss:0.06811
[172]	train-logloss:0.01474	eval-logloss:0.06816
[173]	train-logloss:0.01468	eval-logloss:0.06829
[174]	train-logloss:0.01461	eval-logloss:0.06803
[175]	train-logloss:0.01454	eval-logloss:0.06794
[176]	train-logloss:0.01449	eval-logloss:0.06791
[177]	train-logloss:0.01444	eval-logloss:0.06797
[178]	train-logloss:0.01436	eval-logloss:0.06787
[179]	train-logloss:0.01430	eval-logloss:0.06790
[180]	train-logloss:0.01424	eval-logloss:0.06795
[181]	train-logloss:0.01419	eval-logloss:0.06795
[182]	train-logloss:0.01414	eval-logloss:0.06777
[183]	train-logloss:0.01410	eval-logloss:0.06780
[184]	train-logloss:0.01402	eval-logloss:0.06771
[185]	train-logloss:0.01399	eval-logloss:0.06781
[186]	train-logloss:0.01395	eval-logloss:0.06784
[187]	train-logloss:0.01389	eval-logloss:0.06764
[188]	train-logloss:0.01383	eval-logloss:0.06753
[189]	train-logloss:0.01379	eval-logloss:0.06739
[190]	train-logloss:

[338]	train-logloss:0.00990	eval-logloss:0.06493
[339]	train-logloss:0.00990	eval-logloss:0.06489
[340]	train-logloss:0.00988	eval-logloss:0.06490
[341]	train-logloss:0.00987	eval-logloss:0.06499
[342]	train-logloss:0.00986	eval-logloss:0.06498
[343]	train-logloss:0.00985	eval-logloss:0.06493
[344]	train-logloss:0.00984	eval-logloss:0.06501
[345]	train-logloss:0.00984	eval-logloss:0.06497
[346]	train-logloss:0.00982	eval-logloss:0.06503
[347]	train-logloss:0.00981	eval-logloss:0.06501
[348]	train-logloss:0.00980	eval-logloss:0.06495
[349]	train-logloss:0.00979	eval-logloss:0.06495
[350]	train-logloss:0.00977	eval-logloss:0.06506
[351]	train-logloss:0.00975	eval-logloss:0.06495
[352]	train-logloss:0.00974	eval-logloss:0.06493
[353]	train-logloss:0.00972	eval-logloss:0.06501
[354]	train-logloss:0.00972	eval-logloss:0.06503
[355]	train-logloss:0.00970	eval-logloss:0.06510
[356]	train-logloss:0.00969	eval-logloss:0.06495
[357]	train-logloss:0.00968	eval-logloss:0.06496
[358]	train-logloss:

[506]	train-logloss:0.00846	eval-logloss:0.06374
[507]	train-logloss:0.00845	eval-logloss:0.06380
[508]	train-logloss:0.00845	eval-logloss:0.06381
[509]	train-logloss:0.00845	eval-logloss:0.06385
[510]	train-logloss:0.00843	eval-logloss:0.06376
[511]	train-logloss:0.00842	eval-logloss:0.06372
[512]	train-logloss:0.00842	eval-logloss:0.06374
[513]	train-logloss:0.00842	eval-logloss:0.06367
[514]	train-logloss:0.00842	eval-logloss:0.06372
[515]	train-logloss:0.00841	eval-logloss:0.06367
[516]	train-logloss:0.00840	eval-logloss:0.06373
[517]	train-logloss:0.00839	eval-logloss:0.06382
[518]	train-logloss:0.00838	eval-logloss:0.06369
[519]	train-logloss:0.00838	eval-logloss:0.06367
[520]	train-logloss:0.00837	eval-logloss:0.06375
[521]	train-logloss:0.00837	eval-logloss:0.06377
[522]	train-logloss:0.00836	eval-logloss:0.06384
[523]	train-logloss:0.00836	eval-logloss:0.06381
[524]	train-logloss:0.00835	eval-logloss:0.06382
[525]	train-logloss:0.00835	eval-logloss:0.06386
[526]	train-logloss:

[674]	train-logloss:0.00785	eval-logloss:0.06357
[675]	train-logloss:0.00785	eval-logloss:0.06349
[676]	train-logloss:0.00784	eval-logloss:0.06352
[677]	train-logloss:0.00784	eval-logloss:0.06351
[678]	train-logloss:0.00784	eval-logloss:0.06355
[679]	train-logloss:0.00784	eval-logloss:0.06352
[680]	train-logloss:0.00784	eval-logloss:0.06357
[681]	train-logloss:0.00783	eval-logloss:0.06358
[682]	train-logloss:0.00783	eval-logloss:0.06356
[683]	train-logloss:0.00783	eval-logloss:0.06357
[684]	train-logloss:0.00783	eval-logloss:0.06355
[685]	train-logloss:0.00782	eval-logloss:0.06350
[686]	train-logloss:0.00782	eval-logloss:0.06351
[687]	train-logloss:0.00782	eval-logloss:0.06349
Training Accuracy: 100.00%
Test Accuracy: 100.00%


In [4]:
# Hzごと
# 特徴量と目標変数を抽出する
X = df.drop('State', axis=1)  # 'State'以外の列すべてを特徴量とします
y = df['State']  # 'State'を目標変数とします

#XGBoost
XGBoost(X, y)

[0]	train-logloss:0.62607	eval-logloss:0.62860
[1]	train-logloss:0.57061	eval-logloss:0.57635
[2]	train-logloss:0.53808	eval-logloss:0.54454
[3]	train-logloss:0.50265	eval-logloss:0.50944
[4]	train-logloss:0.47776	eval-logloss:0.48529
[5]	train-logloss:0.44554	eval-logloss:0.45536
[6]	train-logloss:0.42836	eval-logloss:0.43837
[7]	train-logloss:0.39822	eval-logloss:0.40962
[8]	train-logloss:0.37177	eval-logloss:0.38361
[9]	train-logloss:0.35634	eval-logloss:0.36860
[10]	train-logloss:0.33371	eval-logloss:0.34693
[11]	train-logloss:0.32142	eval-logloss:0.33505
[12]	train-logloss:0.30268	eval-logloss:0.31951
[13]	train-logloss:0.29109	eval-logloss:0.30886
[14]	train-logloss:0.28392	eval-logloss:0.30206
[15]	train-logloss:0.27490	eval-logloss:0.29338
[16]	train-logloss:0.25667	eval-logloss:0.27676
[17]	train-logloss:0.24831	eval-logloss:0.26928
[18]	train-logloss:0.23380	eval-logloss:0.25667
[19]	train-logloss:0.22544	eval-logloss:0.24807
[20]	train-logloss:0.22019	eval-logloss:0.24341
[2

[170]	train-logloss:0.00584	eval-logloss:0.01296
[171]	train-logloss:0.00578	eval-logloss:0.01276
[172]	train-logloss:0.00572	eval-logloss:0.01265
[173]	train-logloss:0.00569	eval-logloss:0.01263
[174]	train-logloss:0.00562	eval-logloss:0.01259
[175]	train-logloss:0.00559	eval-logloss:0.01259
[176]	train-logloss:0.00554	eval-logloss:0.01253
[177]	train-logloss:0.00549	eval-logloss:0.01243
[178]	train-logloss:0.00546	eval-logloss:0.01239
[179]	train-logloss:0.00543	eval-logloss:0.01233
[180]	train-logloss:0.00537	eval-logloss:0.01223
[181]	train-logloss:0.00532	eval-logloss:0.01221
[182]	train-logloss:0.00528	eval-logloss:0.01221
[183]	train-logloss:0.00524	eval-logloss:0.01217
[184]	train-logloss:0.00518	eval-logloss:0.01214
[185]	train-logloss:0.00515	eval-logloss:0.01206
[186]	train-logloss:0.00511	eval-logloss:0.01200
[187]	train-logloss:0.00507	eval-logloss:0.01194
[188]	train-logloss:0.00504	eval-logloss:0.01190
[189]	train-logloss:0.00501	eval-logloss:0.01184
[190]	train-logloss:

[337]	train-logloss:0.00285	eval-logloss:0.00822
[338]	train-logloss:0.00285	eval-logloss:0.00821
[339]	train-logloss:0.00284	eval-logloss:0.00822
[340]	train-logloss:0.00284	eval-logloss:0.00822
[341]	train-logloss:0.00284	eval-logloss:0.00818
[342]	train-logloss:0.00283	eval-logloss:0.00815
[343]	train-logloss:0.00283	eval-logloss:0.00817
[344]	train-logloss:0.00283	eval-logloss:0.00813
[345]	train-logloss:0.00282	eval-logloss:0.00813
[346]	train-logloss:0.00282	eval-logloss:0.00816
[347]	train-logloss:0.00281	eval-logloss:0.00816
[348]	train-logloss:0.00281	eval-logloss:0.00817
[349]	train-logloss:0.00280	eval-logloss:0.00816
[350]	train-logloss:0.00280	eval-logloss:0.00814
[351]	train-logloss:0.00279	eval-logloss:0.00809
[352]	train-logloss:0.00279	eval-logloss:0.00808
[353]	train-logloss:0.00279	eval-logloss:0.00810
[354]	train-logloss:0.00278	eval-logloss:0.00811
[355]	train-logloss:0.00277	eval-logloss:0.00810
[356]	train-logloss:0.00277	eval-logloss:0.00810
[357]	train-logloss:

[505]	train-logloss:0.00235	eval-logloss:0.00728
[506]	train-logloss:0.00234	eval-logloss:0.00728
[507]	train-logloss:0.00234	eval-logloss:0.00726
[508]	train-logloss:0.00233	eval-logloss:0.00725
[509]	train-logloss:0.00233	eval-logloss:0.00725
[510]	train-logloss:0.00233	eval-logloss:0.00724
[511]	train-logloss:0.00233	eval-logloss:0.00723
[512]	train-logloss:0.00233	eval-logloss:0.00722
[513]	train-logloss:0.00232	eval-logloss:0.00722
[514]	train-logloss:0.00232	eval-logloss:0.00722
[515]	train-logloss:0.00232	eval-logloss:0.00722
[516]	train-logloss:0.00232	eval-logloss:0.00720
[517]	train-logloss:0.00231	eval-logloss:0.00719
[518]	train-logloss:0.00231	eval-logloss:0.00720
[519]	train-logloss:0.00231	eval-logloss:0.00720
[520]	train-logloss:0.00231	eval-logloss:0.00719
[521]	train-logloss:0.00231	eval-logloss:0.00719
[522]	train-logloss:0.00231	eval-logloss:0.00720
[523]	train-logloss:0.00230	eval-logloss:0.00720
[524]	train-logloss:0.00230	eval-logloss:0.00719
[525]	train-logloss:

[673]	train-logloss:0.00210	eval-logloss:0.00695
[674]	train-logloss:0.00210	eval-logloss:0.00695
[675]	train-logloss:0.00210	eval-logloss:0.00695
[676]	train-logloss:0.00209	eval-logloss:0.00695
[677]	train-logloss:0.00209	eval-logloss:0.00694
[678]	train-logloss:0.00209	eval-logloss:0.00694
[679]	train-logloss:0.00209	eval-logloss:0.00694
[680]	train-logloss:0.00209	eval-logloss:0.00694
[681]	train-logloss:0.00209	eval-logloss:0.00695
[682]	train-logloss:0.00209	eval-logloss:0.00695
[683]	train-logloss:0.00209	eval-logloss:0.00696
[684]	train-logloss:0.00209	eval-logloss:0.00695
[685]	train-logloss:0.00209	eval-logloss:0.00696
[686]	train-logloss:0.00209	eval-logloss:0.00695
[687]	train-logloss:0.00209	eval-logloss:0.00696
[688]	train-logloss:0.00208	eval-logloss:0.00696
[689]	train-logloss:0.00208	eval-logloss:0.00695
[690]	train-logloss:0.00208	eval-logloss:0.00696
[691]	train-logloss:0.00208	eval-logloss:0.00695
[692]	train-logloss:0.00208	eval-logloss:0.00695
[693]	train-logloss:

...

[1500]	train-logloss:0.00183	eval-logloss:0.00659
[1501]	train-logloss:0.00183	eval-logloss:0.00659
[1502]	train-logloss:0.00183	eval-logloss:0.00659
[1503]	train-logloss:0.00183	eval-logloss:0.00659
[1504]	train-logloss:0.00183	eval-logloss:0.00659
[1505]	train-logloss:0.00183	eval-logloss:0.00659
[1506]	train-logloss:0.00183	eval-logloss:0.00659
[1507]	train-logloss:0.00183	eval-logloss:0.00658
[1508]	train-logloss:0.00183	eval-logloss:0.00658
[1509]	train-logloss:0.00183	eval-logloss:0.00658
[1510]	train-logloss:0.00183	eval-logloss:0.00659
[1511]	train-logloss:0.00183	eval-logloss:0.00659
[1512]	train-logloss:0.00183	eval-logloss:0.00659
[1513]	train-logloss:0.00183	eval-logloss:0.00659
[1514]	train-logloss:0.00183	eval-logloss:0.00659
[1515]	train-logloss:0.00183	eval-logloss:0.00659
[1516]	train-logloss:0.00183	eval-logloss:0.00659
[1517]	train-logloss:0.00183	eval-logloss:0.00659
[1518]	train-logloss:0.00183	eval-logloss:0.00659
[1519]	train-logloss:0.00183	eval-logloss:0.00659


[1664]	train-logloss:0.00180	eval-logloss:0.00657
[1665]	train-logloss:0.00180	eval-logloss:0.00657
[1666]	train-logloss:0.00180	eval-logloss:0.00656
[1667]	train-logloss:0.00180	eval-logloss:0.00656
[1668]	train-logloss:0.00180	eval-logloss:0.00657
[1669]	train-logloss:0.00180	eval-logloss:0.00656
[1670]	train-logloss:0.00180	eval-logloss:0.00656
[1671]	train-logloss:0.00180	eval-logloss:0.00656
[1672]	train-logloss:0.00180	eval-logloss:0.00656
[1673]	train-logloss:0.00180	eval-logloss:0.00656
[1674]	train-logloss:0.00180	eval-logloss:0.00656
[1675]	train-logloss:0.00180	eval-logloss:0.00656
[1676]	train-logloss:0.00180	eval-logloss:0.00656
[1677]	train-logloss:0.00180	eval-logloss:0.00656
[1678]	train-logloss:0.00180	eval-logloss:0.00656
[1679]	train-logloss:0.00180	eval-logloss:0.00656
[1680]	train-logloss:0.00180	eval-logloss:0.00657
[1681]	train-logloss:0.00180	eval-logloss:0.00656
[1682]	train-logloss:0.00180	eval-logloss:0.00657
[1683]	train-logloss:0.00180	eval-logloss:0.00657


[1827]	train-logloss:0.00178	eval-logloss:0.00655
[1828]	train-logloss:0.00178	eval-logloss:0.00655
[1829]	train-logloss:0.00178	eval-logloss:0.00655
[1830]	train-logloss:0.00178	eval-logloss:0.00656
[1831]	train-logloss:0.00178	eval-logloss:0.00655
[1832]	train-logloss:0.00178	eval-logloss:0.00655
[1833]	train-logloss:0.00178	eval-logloss:0.00655
[1834]	train-logloss:0.00178	eval-logloss:0.00655
[1835]	train-logloss:0.00178	eval-logloss:0.00655
[1836]	train-logloss:0.00178	eval-logloss:0.00655
[1837]	train-logloss:0.00178	eval-logloss:0.00655
[1838]	train-logloss:0.00178	eval-logloss:0.00655
[1839]	train-logloss:0.00178	eval-logloss:0.00655
[1840]	train-logloss:0.00178	eval-logloss:0.00655
[1841]	train-logloss:0.00178	eval-logloss:0.00655
[1842]	train-logloss:0.00178	eval-logloss:0.00655
[1843]	train-logloss:0.00178	eval-logloss:0.00655
[1844]	train-logloss:0.00178	eval-logloss:0.00655
[1845]	train-logloss:0.00178	eval-logloss:0.00655
[1846]	train-logloss:0.00178	eval-logloss:0.00655


[1991]	train-logloss:0.00176	eval-logloss:0.00654
[1992]	train-logloss:0.00176	eval-logloss:0.00654
[1993]	train-logloss:0.00176	eval-logloss:0.00654
[1994]	train-logloss:0.00176	eval-logloss:0.00654
[1995]	train-logloss:0.00176	eval-logloss:0.00654
[1996]	train-logloss:0.00176	eval-logloss:0.00654
[1997]	train-logloss:0.00176	eval-logloss:0.00654
[1998]	train-logloss:0.00176	eval-logloss:0.00654
[1999]	train-logloss:0.00176	eval-logloss:0.00653
[2000]	train-logloss:0.00176	eval-logloss:0.00654
[2001]	train-logloss:0.00176	eval-logloss:0.00653
[2002]	train-logloss:0.00176	eval-logloss:0.00653
[2003]	train-logloss:0.00176	eval-logloss:0.00653
[2004]	train-logloss:0.00176	eval-logloss:0.00653
[2005]	train-logloss:0.00176	eval-logloss:0.00652
[2006]	train-logloss:0.00176	eval-logloss:0.00653
[2007]	train-logloss:0.00176	eval-logloss:0.00653
[2008]	train-logloss:0.00176	eval-logloss:0.00653
[2009]	train-logloss:0.00176	eval-logloss:0.00653
[2010]	train-logloss:0.00176	eval-logloss:0.00653


[2155]	train-logloss:0.00174	eval-logloss:0.00653
[2156]	train-logloss:0.00174	eval-logloss:0.00653
[2157]	train-logloss:0.00174	eval-logloss:0.00653
[2158]	train-logloss:0.00174	eval-logloss:0.00653
[2159]	train-logloss:0.00174	eval-logloss:0.00653
[2160]	train-logloss:0.00174	eval-logloss:0.00654
[2161]	train-logloss:0.00174	eval-logloss:0.00653
[2162]	train-logloss:0.00174	eval-logloss:0.00653
[2163]	train-logloss:0.00174	eval-logloss:0.00654
[2164]	train-logloss:0.00174	eval-logloss:0.00653
[2165]	train-logloss:0.00174	eval-logloss:0.00653
[2166]	train-logloss:0.00174	eval-logloss:0.00653
[2167]	train-logloss:0.00174	eval-logloss:0.00653
[2168]	train-logloss:0.00174	eval-logloss:0.00653
[2169]	train-logloss:0.00174	eval-logloss:0.00653
[2170]	train-logloss:0.00174	eval-logloss:0.00653
[2171]	train-logloss:0.00174	eval-logloss:0.00653
[2172]	train-logloss:0.00174	eval-logloss:0.00654
[2173]	train-logloss:0.00174	eval-logloss:0.00653
[2174]	train-logloss:0.00174	eval-logloss:0.00654
